In [1]:
from google.colab import drive
import struct
from struct import unpack
import os
from os import listdir
from tensorflow.keras.models import load_model
from PIL import Image, ImageDraw
import numpy as np
import random
import math
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten

# colors used for each stroke. there are 120 colors here
colors = [(166, 98, 151), (113, 17, 39), (9, 241, 137), (101, 57, 169), (83, 114, 65),
          (148, 192, 218), (33, 244, 163), (227, 161, 190), (0, 132, 34), (215, 0, 34),
          (62, 153, 156), (27, 175, 51), (195, 177, 129), (162, 50, 31), (111, 145, 161),
          (71, 112, 92), (207, 34, 125), (42, 239, 132), (173, 186, 111), (146, 47, 65),
          (20, 232, 202), (142, 39, 235), (124, 92, 27), (196, 181, 151), (3, 31, 18),
          (31, 62, 58), (170, 86, 63), (126, 128, 206), (25, 215, 6), (76, 7, 231),
          (185, 69, 255), (255, 84, 155), (91, 91, 56), (103, 40, 85), (129, 32, 126),
          (157, 111, 1), (111, 236, 105), (185, 201, 217), (50, 74, 161), (83, 149, 229),
          (62, 3, 139), (105, 28, 157), (85, 166, 35), (49, 216, 177), (102, 104, 20),
          (239, 73, 245), (224, 206, 121), (205, 237, 233), (159, 150, 10), (171, 34, 210),
          (37, 90, 200), (230, 220, 191), (65, 109, 204), (142, 242, 24), (87, 68, 167),
          (30, 53, 30), (31, 48, 70), (211, 47, 108), (83, 203, 203), (152, 236, 213),
          (111, 204, 214), (205, 166, 70), (141, 40, 198), (239, 25, 181), (205, 113, 144),
          (146, 123, 143), (129, 156, 82), (164, 137, 25), (95, 172, 97), (228, 105, 172),
          (53, 126, 182), (173, 4, 88), (236, 95, 71), (184, 108, 90), (169, 113, 95),
          (184, 216, 247), (35, 104, 23), (23, 211, 201), (161, 235, 196), (61, 32, 111),
          (148, 200, 40), (90, 69, 122), (156, 119, 67), (44, 11, 48), (182, 174, 253),
          (255, 18, 251), (17, 81, 203), (46, 238, 243), (218, 221, 119), (205, 190, 157), 
          (192, 17, 27), (71, 19, 122), (158, 9, 63), (206, 116, 96), (162, 215, 232),
          (125, 94, 222), (117, 179, 132), (208, 193, 98), (161, 111, 151), (254, 54, 178),
          (103, 44, 188), (78, 121, 137), (193, 161, 140), (234, 173, 216), (252, 169, 79),
          (167, 0, 56), (103, 136, 238), (108, 175, 151), (85, 228, 221), (192, 48, 46), 
          (206, 146, 63), (250, 126, 4), (91, 8, 55), (13, 55, 186), (191, 24, 109),
          (160, 46, 155), (194, 202, 22), (107, 6, 39), (8, 71, 88), (9, 168, 161)]

# number of images per category
run = 1
STROKE_THRESHOLD = 120
N_SAMPLES = 800
IMAGE_THRESHOLD = (run - 1) * N_SAMPLES
TRAIN_RATIO = .96
N_CATEGORIES = 220

In [2]:
# connect drive to colab and switch to the work folder
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/Colab

Mounted at /content/drive
/content/drive/MyDrive/Colab


In [3]:
# if the data folder is not present, create it, otherwise pass
if os.path.exists('data') is False:
  !mkdir data
  !gsutil -m cp 'gs://quickdraw_dataset/full/binary/*.bin' data2
  print("directory created")
else:
  print("directory already exists")

directory already exists


Download the *binary* files into the newly created `data` directory. We are downloading the *binary* version of the dataset to save space.

Reading the binary data as per the official documentation.

In [4]:
def unpack_drawing(file_handle):
    # image id, country code, whether it was recognized or not and timestamp are
    #  ignored
    _, = unpack('Q', file_handle.read(8))
    _, = unpack('2s', file_handle.read(2))
    _, = unpack('b', file_handle.read(1))
    _, = unpack('I', file_handle.read(4))
    
    n_strokes, = unpack('H', file_handle.read(2))
    drawing = []
    for i in range(n_strokes):
        n_points, = unpack('H', file_handle.read(2))
        fmt = str(n_points) + 'B'
        x = unpack(fmt, file_handle.read(n_points))
        y = unpack(fmt, file_handle.read(n_points))
        drawing.append((x, y))
 
    return drawing

In [5]:
def unpack_drawings(filename):
    with open("data/" + filename, 'rb') as f:
        while True:
            try:
                yield unpack_drawing(f)
            except struct.error:
                break

To automate the process of reading the files, the algorithm iterates through the contents of the `data` directory using `listdir`.

In [6]:
# resized image shape
new_size = (32, 32)

def create_image(drawing):
  img = Image.new("RGB", (256, 256))
  # if doodle has inhuman number of strokes, ignore
  if len(drawing) >= STROKE_THRESHOLD:
    return None

  # iterate through strokes
  for index, stroke in enumerate(drawing):
    prev_x, prev_y = None, None
    # iterate through coordinates in a stroke
    for idx in range(len(stroke[0])):
      x = stroke[0][idx]
      y = stroke[1][idx]
      pixels = img.load()
      pixels[x, y] = colors[index]

      # draw line between this and previous coordinates, if not first coordinate
      #  of stroke
      if prev_x is not None and (prev_x != x or prev_y != y):
        shape = [(prev_x, prev_y), (x, y)]
        img1 = ImageDraw.Draw(img)
        img1.line(shape, fill = colors[index], width = 2)

      prev_x, prev_y = x, y
  
  # convert image to numpy array
  img_arr = np.asarray(img.resize(new_size))
  return img_arr

In [7]:
def save_network(model):
  model.save(os.path.join(".", "network.h5"))
  print("Model saved")

def load_network(file_name = "network.h5"):
  return load_model(os.path.join(".", file_name))

def normalize(data):
    return np.interp(data, [0, 255], [-1, 1])

In [8]:
def separate_data(data, category):
  random.shuffle(data)
  data = data[:N_SAMPLES]
  data = normalize(data)

  train_count = math.ceil(N_SAMPLES * TRAIN_RATIO)
  x_train = data[:train_count]
  y_train = [category] * train_count
  
  test_count = N_SAMPLES - train_count
  x_test = data[train_count:]
  y_test = [category] * test_count

  # one hot encoding
  y_train = np_utils.to_categorical(y_train, N_CATEGORIES)
  y_test = np_utils.to_categorical(y_test, N_CATEGORIES)

  return x_train, y_train, x_test, y_test

In [9]:
def unpack_all():
  training_set = []
  training_labels = []
  test_set = []
  test_labels = []
 
  # iterate through the contents of the data directory
  for index, file_name in enumerate(listdir("data")):
    if index >= N_CATEGORIES:
      break
    print(file_name)
    data = []
    count = 0
    for idx, drawing in enumerate(unpack_drawings(file_name)):
      if idx < IMAGE_THRESHOLD:
        continue
      if count >= N_SAMPLES:
        break
      if idx < 74000 and random.random() < 0.4:
        continue
      img_arr = create_image(drawing)
      # if returned value is an Image object, append to data list
      if img_arr is not None:
        count += 1
        data.append(img_arr)
    
    # shuffled lists of images and label lists
    x_train, y_train, x_test, y_test = separate_data(data, index)
    
    training_set.extend(x_train)
    training_labels.extend(y_train)
    test_set.extend(x_test)
    test_labels.extend(y_test)
  
  # when all data is read in, convert lists into numpy arrays
  training_set = np.array(training_set)
  training_labels = np.array(training_labels)
  test_set = np.array(test_set)
  test_labels = np.array(test_labels)

  return training_set, training_labels, test_set, test_labels

def unpack_all_for_testing():
  test_set = []
  test_labels = []
 
  # iterate through the contents of the data directory
  for index, file_name in enumerate(listdir("data")):
    if index >= N_CATEGORIES:
      break
    print(file_name)
    data = []
    count = 0
    for idx, drawing in enumerate(unpack_drawings(file_name)):
      if idx < IMAGE_THRESHOLD:
        continue
      if count >= N_SAMPLES:
        break

      img_arr = create_image(drawing)
      # if returned value is an Image object, append to data list
      if img_arr is not None:
        count += 1
        data.append(img_arr)
    
    y_test = [index] * count
    y_test = np_utils.to_categorical(y_test, N_CATEGORIES)
    data = normalize(data)
    test_set.extend(data)
    test_labels.extend(y_test)
    
  # when all data is read in, convert lists into numpy arrays
  test_set = np.array(test_set)
  test_labels = np.array(test_labels)

  return test_set, test_labels

In [10]:
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(N_CATEGORIES, activation='softmax'))

Compile and save the model. This will overwrite the saved network, use carefully.

In [11]:
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# save_network(model)

In [12]:
training_set, training_labels, test_set, test_labels = unpack_all()

# # shuffle training set and labels in unison
train_idx = np.arange(len(training_set))
np.random.shuffle(train_idx)
training_set = training_set[train_idx]
training_labels = training_labels[train_idx]

# # shuffle test set and labels in unison
test_idx = np.arange(len(test_set))
np.random.shuffle(test_idx)
test_set = test_set[test_idx]
test_labels = test_labels[test_idx]

The Eiffel Tower.bin
aircraft carrier.bin
airplane.bin
angel.bin
alarm clock.bin
anvil.bin
The Great Wall of China.bin
The Mona Lisa.bin
ant.bin
animal migration.bin
ambulance.bin
arm.bin
axe.bin
barn.bin
baseball bat.bin
apple.bin
backpack.bin
bandage.bin
baseball.bin
asparagus.bin
banana.bin
basketball.bin
basket.bin
beach.bin
bed.bin
bathtub.bin
bat.bin
bench.bin
bear.bin
bee.bin
belt.bin
bicycle.bin
bird.bin
beard.bin
binoculars.bin
boomerang.bin
book.bin
birthday cake.bin
blueberry.bin
bowtie.bin
bottlecap.bin
bread.bin
bucket.bin
bridge.bin
blackberry.bin
bracelet.bin
broom.bin
brain.bin
broccoli.bin
bush.bin
butterfly.bin
bulldozer.bin
cactus.bin
bus.bin
calculator.bin
cake.bin
camel.bin
camera.bin
canoe.bin
candle.bin
campfire.bin
cannon.bin
car.bin
castle.bin
calendar.bin
carrot.bin
cell phone.bin
ceiling fan.bin
camouflage.bin
cat.bin
circle.bin
church.bin
cloud.bin
clock.bin
chair.bin
clarinet.bin
compass.bin
cello.bin
coffee cup.bin
chandelier.bin
computer.bin
couch.bin
cra

In [13]:
def train_one_epoch(file_name = "network.h5"):
  model = load_network(file_name)
  print("Started training")
  model.fit(training_set, training_labels, batch_size=32, epochs=20)
  print("Training complete")
  save_network(model)

def predict(file_name = "network.h5"):
  model = load_network(file_name)
  preds = model.predict(test_set)

  score = 0
  for i in range(len(preds)):
      if np.argmax(preds[i]) == np.argmax(test_labels[i]):
          score += 1
          
  print(score)
  print("Accuracy: ", ((score + 0.0) / len(preds)) * 100)

In [ ]:
# test_set, test_labels = unpack_all_for_testing()

# test_idx = np.arange(len(test_set))
# np.random.shuffle(test_idx)
# test_set = test_set[test_idx]
# test_labels = test_labels[test_idx]

# predict()

The Eiffel Tower.bin
aircraft carrier.bin
airplane.bin
angel.bin
alarm clock.bin
anvil.bin
The Great Wall of China.bin
The Mona Lisa.bin
ant.bin
animal migration.bin
ambulance.bin
arm.bin
axe.bin
barn.bin
baseball bat.bin
apple.bin
backpack.bin
bandage.bin
baseball.bin
asparagus.bin
banana.bin
basketball.bin
basket.bin
beach.bin
bed.bin
bathtub.bin
bat.bin
bench.bin
bear.bin
bee.bin
belt.bin
bicycle.bin
bird.bin
beard.bin
binoculars.bin
boomerang.bin
book.bin
birthday cake.bin
blueberry.bin
bowtie.bin
bottlecap.bin
bread.bin
bucket.bin
bridge.bin
blackberry.bin
bracelet.bin
broom.bin
brain.bin
broccoli.bin
bush.bin
butterfly.bin
bulldozer.bin
cactus.bin
bus.bin
calculator.bin
cake.bin
camel.bin
camera.bin
canoe.bin
candle.bin
campfire.bin
cannon.bin
car.bin
castle.bin
calendar.bin
carrot.bin
cell phone.bin
ceiling fan.bin
camouflage.bin
cat.bin
circle.bin
church.bin
cloud.bin
clock.bin
chair.bin
clarinet.bin
compass.bin
cello.bin
coffee cup.bin
chandelier.bin
computer.bin
couch.bin
cra

In [ ]:
# from keras.utils.vis_utils import plot_model

# model = load_network()

# print(model.summary())
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)